In [1]:
from stringcase import snakecase
from urllib.parse import quote
from pathlib import Path
import json
from xml.etree import ElementTree

In [2]:
from robot.libdocpkg import LibraryDocumentation
from robot.libraries import STDLIBS

In [3]:
HERE = Path(".")
BLOCKS = (HERE / ".." / "blocks")
TOOLBOX = (HERE / ".." / "xml")

In [4]:
libdoc = LibraryDocumentation("BuiltIn")
kwdoc = libdoc.keywords[0]

In [5]:
all_blocks = {}
toolbox = ElementTree.parse(str(TOOLBOX / "toolbox.xml")).getroot()

In [6]:
tool_cat = ElementTree.Element('category')
tool_cat.attrib['name'] = 'Robot'
toolbox.append(tool_cat)
tool_block = ElementTree.Element("block")
tool_block.attrib["type"] = "robot___test_case"
tool_cat.append(tool_block)

In [7]:
all_blocks["Robot"] = dict(
    robot___test_case=dict(
        name="Test Case",
        args=[],
        block=dict(
            type="robot___test_case",
            message0=f"Test Case %1",
            args0=[{"type": "input_dummy"}],
            nextStatement=True,
            previousStatement=True,
        )
    )
)

In [8]:
for lib_name in STDLIBS:
    try: libdoc = LibraryDocumentation(lib_name)
    except: continue
   
    tool_cat = ElementTree.Element('category')
    tool_cat.attrib['name'] = lib_name
    toolbox.append(tool_cat)

    snake_name = snakecase(lib_name)
    lib_blocks = all_blocks[lib_name] = {}
    for kwdoc in libdoc.keywords:
        name = f"robot_{snake_name}___{snakecase(kwdoc.name)}"
 
        block = dict(
            tooltip=kwdoc.shortdoc,
            helpUrl=f"http://robotframework.org/robotframework/latest/libraries/{lib_name}.html#{quote(kwdoc.name)}",
            type=name,
            message0=f"{kwdoc.name} %1",
            args0=[{"type": "input_dummy"}],
            nextStatement=True,
            previousStatement=True,
            output='String',
        )

        
        for i, arg in enumerate(kwdoc.args):
            if "*" in arg: continue
            block.update({
                f"message{i + 1}": f"{arg} %1",
                f"args{i + 1}": [{
                    "type": "input_value",
                    "name": arg.split("=")[0]
                }]
            })
        
        lib_blocks[name] = dict(
            name=kwdoc.name,
            block=block,
            args=[
                a.split("=")[0] for a in kwdoc.args
                if "*" not in a
            ],
            template=f"""    {kwdoc.name}""",
        )
        tool_block = ElementTree.Element("block")
        tool_block.attrib["type"] = name
        tool_cat.append(tool_block)
        
        
(BLOCKS / "robot" / "stdlibs.json").write_text(json.dumps(all_blocks, indent=2, sort_keys=True))
(TOOLBOX / "robot.xml").write_bytes(ElementTree.tostring(toolbox))

26619